In [1]:
import QuantLib as ql
from Enums import Currency, SwapDirection
from instruments.interestRateInstrument.interestRateSwap import InterestRateSwap
from instruments.interestRateInstrument.swaption import Swaption
from marketdata import init_marketdata

In [4]:
irs = InterestRateSwap(notional = 1000000, 
                       currency = Currency.USD, 
                       timeToSwapStart=2, 
                       timeToSwapEnd=11, 
                       swapDirection=SwapDirection.RECEIVER,
                       fixed_rate=0.025, 
                       float_spread = 0.004)

In [10]:
exerciseDate = init_marketdata.today + ql.Period(1, ql.Years)
exercise = ql.EuropeanExercise(exerciseDate)
swaption = ql.Swaption(irs.ql_swap, exercise)

In [12]:
from marketdata.util import today, calendar, day_count, business_day_convention
yts = ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(0.02)), day_count)
yts_handle = ql.YieldTermStructureHandle(yts)

Pricing swaption using Black 76 and a single quote volatility

In [ ]:
vol_handle = ql.QuoteHandle(ql.SimpleQuote(0.2))
single_quote_engine = ql.BlackSwaptionEngine(yts_handle, vol_handle)

In [14]:
swaption.setPricingEngine(single_quote_engine)

In [15]:
swaption.NPV()

15971.59884786479

Pricing swaption using Black 76 and a ``ConstantSwaptionVolatility``

In [27]:
flat_surface_handle = ql.SwaptionVolatilityStructureHandle(ql.ConstantSwaptionVolatility(today, calendar, business_day_convention, ql.QuoteHandle(ql.SimpleQuote(0.2)), day_count))
flat_surface_engine = ql.BlackSwaptionEngine(yts_handle, flat_surface_handle)

In [28]:
swaption.setPricingEngine(flat_surface_engine)
swaption.NPV()

15971.59884786479

Pricing swaption using Black 76 and a Swaption Vol Surface